#1.Lectura de los Datos


In [ ]:
pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np

#1.lectura del dataset de entrenamiento
df_Rstate_train=pd.read_csv("X_train.csv")
df_Rstate_train

#2.lectura del dataset de Test
df_Rstate_test=pd.read_csv("X_test.csv")
df_Rstate_test

#lectura del dataset "Y" de entrenamiento
df_y_train=pd.read_csv("y_train.csv")
df_y_train.drop("id_annonce", axis=1, inplace=True)
df_y_train

#lectura del dataset "Y" de test
df_y_test=pd.read_csv("y_random.csv")
df_y_test.drop("id_annonce", axis=1, inplace=True)
df_y_test

df_y_train.mean()

,0
price,343221.272051


In [ ]:
#3.Informacion del dataset de entrenamiento

df_Rstate_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37368 entries, 0 to 37367
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id_annonce                   37368 non-null  int64  
 1   property_type                37368 non-null  object 
 2   approximate_latitude         37368 non-null  float64
 3   approximate_longitude        37368 non-null  float64
 4   city                         37368 non-null  object 
 5   postal_code                  37368 non-null  int64  
 6   size                         36856 non-null  float64
 7   floor                        9743 non-null   float64
 8   land_size                    15581 non-null  float64
 9   energy_performance_value     19068 non-null  float64
 10  energy_performance_category  19068 non-null  object 
 11  ghg_value                    18530 non-null  float64
 12  ghg_category                 18530 non-null  object 
 13  exposition      

In [ ]:
#4.informacion del dataset de test

df_Rstate_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9339 entries, 0 to 9338
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id_annonce                   9339 non-null   int64  
 1   property_type                9339 non-null   object 
 2   approximate_latitude         9339 non-null   float64
 3   approximate_longitude        9339 non-null   float64
 4   city                         9339 non-null   object 
 5   postal_code                  9339 non-null   int64  
 6   size                         9203 non-null   float64
 7   floor                        2485 non-null   float64
 8   land_size                    3852 non-null   float64
 9   energy_performance_value     4828 non-null   float64
 10  energy_performance_category  4828 non-null   object 
 11  ghg_value                    4679 non-null   float64
 12  ghg_category                 4679 non-null   object 
 13  exposition        

In [ ]:
#5. Separar las variables numericas del dataset de Entrenamiento y de Test

df_Rstate_train_num = df_Rstate_train.loc[:, ['approximate_latitude',
                                              'approximate_longitude',
                                              'size','floor','land_size',
                                              'energy_performance_value',
                                              'ghg_value','nb_rooms','nb_bedrooms','nb_bathrooms',
                                              'nb_parking_places','nb_boxes',]].copy()

df_Rstate_test_num = df_Rstate_test.loc[:, ['approximate_latitude',
                                              'approximate_longitude',
                                              'size','floor','land_size',
                                              'energy_performance_value',
                                              'ghg_value','nb_rooms','nb_bedrooms','nb_bathrooms',
                                              'nb_parking_places','nb_boxes',]].copy()

In [ ]:
df_Rstate_train_num.shape, df_Rstate_test_num.shape, df_y_train.shape

((37368, 12), (9339, 12), (37368, 1))

In [ ]:
#imputamos los valores faltantes con la media para cada columna numerica

df_Rstate_train_num = df_Rstate_train_num.fillna(df_Rstate_train_num.median(numeric_only=True))
df_Rstate_test_num = df_Rstate_test_num.fillna(df_Rstate_test_num.median(numeric_only=True))



In [ ]:
#Estandarizacion de los Datos tanto X como Y

from sklearn.preprocessing import StandardScaler

# Crear el objeto escalador
scaler = StandardScaler()

# Ajustar SOLO con el entrenamiento y transformar
X_train_scaled = scaler.fit_transform(df_Rstate_train_num)

# Transformar los datos de prueba usando los parámetros calculados en train
X_test_scaled = scaler.transform(df_Rstate_test_num )

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(df_y_train.values.reshape(-1,1))

y_test_scaled = y_scaler.transform(df_y_test.values.reshape(-1,1))

In [ ]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=df_Rstate_train_num.columns)
y_train_scaled = pd.DataFrame(y_train_scaled, columns=df_y_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=df_Rstate_test_num.columns)
y_test_scaled = pd.DataFrame(y_test_scaled, columns=df_y_test.columns)

X_train_scaled.shape, y_train_scaled.shape, X_test_scaled.shape, y_test_scaled.shape

((37368, 12), (37368, 1), (9339, 12), (9339, 1))

#2.Encontrar las Variables mas Correlacionadas


In [ ]:
import pandas as pd

# Unimos los datasets X_train_scaled y y_train_sacled  temporalmente para ver las variables mas correlacionadas
df_correlacion = X_train_scaled.copy()
df_correlacion["Price"] = y_train_scaled

# Calculamos correlación
corr = df_correlacion.corr(numeric_only=True)["Price"].sort_values(ascending=False)

print(corr)

Price                       1.000000
nb_rooms                    0.303781
nb_bedrooms                 0.288266
nb_bathrooms                0.172911
approximate_longitude       0.095404
floor                       0.076731
nb_parking_places           0.064793
land_size                   0.037572
approximate_latitude        0.034801
size                        0.034754
nb_boxes                    0.030988
ghg_value                  -0.010407
energy_performance_value   -0.021864
Name: Price, dtype: float64


In [ ]:
from sklearn.model_selection import train_test_split
#Creo el dataset solo con las 3 variables mas correlacionadas y seran los dataframes que se usaran como dataset de la red neuronal

X_train=X_train_scaled[["nb_rooms","nb_bedrooms","nb_bathrooms"]]
X_test=X_test_scaled[["nb_rooms","nb_bedrooms","nb_bathrooms"]]
y_train=y_train_scaled
y_test=y_test_scaled

#Del dataset de entrenamiento extraigo una parte del dataset para validacion
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.20, random_state=42)


print('Tamaño del dataset de Entrenamiento ', X_train.shape, y_train.shape)
print('Tamaño del dataset de Validacion ', X_val.shape,y_val.shape)
print('Tamaño del dataset de Test ', X_test.shape,y_test.shape)




Tamaño del dataset de Entrenamiento  (29894, 3) (29894, 1)
Tamaño del dataset de Validacion  (7474, 3) (7474, 1)
Tamaño del dataset de Test  (9339, 3) (9339, 1)


#3.Definicion de la red Neuronal con regresion lineal


In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

#defino una funcion para crear las redes neuronales


def build_model_regresion(lr_var,neuronas,drop_out,input_data):
    model_Rstate = tf.keras.models.Sequential([
                    #1.primera capa de entrada con dropout de 20%
                    keras.layers.Dense(neuronas, activation='relu', input_shape=(input_data,)),
                    keras.layers.Dropout(drop_out),

                    #2.segunda capa con dropout de 20%
                    keras.layers.Dense(neuronas, activation='relu'),
                    #3.tercera capa de salida
                    keras.layers.Dense(1)])

    model_Rstate.compile(
          optimizer=Adam(learning_rate=lr_var),
          loss='mse',         # error cuadrático medio
          metrics=['mae'] )   # error absoluto medio

    return model_Rstate

In [ ]:
#Creo la red neuronal con los parametros entrenables
#los hiperparametros en este caso serian learning_rate,numero de neuronas de cada capa , porcentaje de dropout de la red , entrada de datos

RN_Regresion=build_model_regresion(0.001,16,0.20,3)
RN_Regresion.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 16)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 353 (1.38 KB)

 Trainable params: 353 (1.38 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#Ejecuto la red Neuronal y guardo su historial

History=RN_Regresion.fit(X_train,
                         y_train,
                         epochs=100,
                         batch_size=512,
                         validation_data=(X_val,y_val))

Epoch 1/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 1.0261 - mae: 0.6300 - val_loss: 0.8948 - val_mae: 0.6266
Epoch 2/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9402 - mae: 0.6350 - val_loss: 0.8874 - val_mae: 0.6250
Epoch 3/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9266 - mae: 0.6257 - val_loss: 0.8867 - val_mae: 0.6273
Epoch 4/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9049 - mae: 0.6240 - val_loss: 0.8824 - val_mae: 0.6231
Epoch 5/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9020 - mae: 0.6246 - val_loss: 0.8751 - val_mae: 0.6185
Epoch 6/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8689 - mae: 0.6124 - val_loss: 0.8765 - val_mae: 0.6248
Epoch 7/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8707 - mae: 0.6166 - val_loss: 0.8744 - val_mae: 0.6230
Epoch 8/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8791 - mae: 0.6205 - val_loss: 0.8738 - val_mae: 0.6213
Epoch 9/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.889

In [ ]:
History.history

{'loss': [0.9284992814064026,
  0.8872761130332947,
  0.8875582218170166,
  0.8849360942840576,
  0.8824700117111206,
  0.8817809820175171,
  0.8840203285217285,
  0.8822095394134521,
  0.8835544586181641,
  0.8786989450454712,
  0.8805490732192993,
  0.8803931474685669,
  0.8773631453514099,
  0.8783050179481506,
  0.8789458870887756,
  0.878038227558136,
  0.877951979637146,
  0.8778710961341858,
  0.877805769443512,
  0.8774493932723999,
  0.8772023916244507,
  0.8775672912597656,
  0.8767704367637634,
  0.8775336742401123,
  0.8771762847900391,
  0.8758292198181152,
  0.8760274648666382,
  0.8768693804740906,
  0.877671480178833,
  0.8757364749908447,
  0.8767946362495422,
  0.8761048913002014,
  0.877487301826477,
  0.877424955368042,
  0.87491375207901,
  0.8757229447364807,
  0.877069890499115,
  0.8762994408607483,
  0.8762574791908264,
  0.8768496513366699,
  0.8758051991462708,
  0.8746597766876221,
  0.8755583763122559,
  0.8744075894355774,
  0.8758797645568848,
  0.8755797

In [ ]:
RN_Regresion.evaluate(X_test,y_test)

292/292 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.1569 - mae: 0.8949


[1.178137183189392, 0.9037420153617859]

#4.Regresion Lineal Polinomial

In [ ]:
#Transformo el dataset en un dataset polinomial

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)

X_train_poly=poly.fit_transform(X_train)
X_test_poly=poly.fit_transform(X_test)
X_val_poly=poly.fit_transform(X_val)

X_train_poly.shape, X_test_poly.shape, X_val_poly.shape

((29894, 10), (9339, 10), (7474, 10))

In [ ]:
#Creo la red neuronal con el dataset polinomial grado 2

RN_Regresion_poly2=build_model_regresion(0.001,128,0.20,10)
RN_Regresion_poly2.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,049 (70.50 KB)

 Trainable params: 18,049 (70.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
History2=RN_Regresion_poly2.fit(X_train_poly,
                         y_train,
                         epochs=100,
                         batch_size=512,
                         validation_data=(X_val_poly,y_val))

Epoch 1/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 1.4811 - mae: 0.6655 - val_loss: 8.9676 - val_mae: 0.6772
Epoch 2/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 1.5872 - mae: 0.6623 - val_loss: 0.9680 - val_mae: 0.6196
Epoch 3/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 1.6636 - mae: 0.6379 - val_loss: 0.9211 - val_mae: 0.6584
Epoch 4/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 3.0695 - mae: 0.6739 - val_loss: 0.9724 - val_mae: 0.6236
Epoch 5/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 3.8619 - mae: 0.6364 - val_loss: 11.1017 - val_mae: 0.7000
Epoch 6/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.4307 - mae: 0.6552 - val_loss: 1.0892 - val_mae: 0.6302
Epoch 7/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 2.7387 - mae: 0.6353 - val_loss: 5.7269 - val_mae: 0.6883
Epoch 8/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.1608 - mae: 0.6671 - val_loss: 0.9203 - val_mae: 0.6226
Epoch 9/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - lo

In [ ]:
RN_Regresion_poly2.evaluate(X_test_poly,y_test)

292/292 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3019 - mae: 0.9152


[1.2680177688598633, 0.9214913249015808]

#5.Regresion Polinomial Grado 3


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3)

X_train_poly3=poly.fit_transform(X_train)
X_test_poly3=poly.fit_transform(X_test)
X_val_poly3=poly.fit_transform(X_val)

X_train_poly3.shape, X_test_poly3.shape, X_val_poly3.shape

((29894, 20), (9339, 20), (7474, 20))

In [ ]:
RN_Regresion_poly3=build_model_regresion(0.001,16,0.20,20)
RN_Regresion_poly3.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 16)             │           336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 625 (2.44 KB)

 Trainable params: 625 (2.44 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
History3=RN_Regresion_poly3.fit(X_train_poly3,
                         y_train,
                         epochs=100,
                         batch_size=32,
                         validation_data=(X_val_poly3,y_val))

Epoch 1/100
935/935 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 576.0444 - mae: 1.0998 - val_loss: 6254.7710 - val_mae: 1.8869
Epoch 2/100
935/935 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 932.2343 - mae: 1.1879 - val_loss: 2814.2839 - val_mae: 1.3587
Epoch 3/100
935/935 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 644.3853 - mae: 1.0361 - val_loss: 115.8536 - val_mae: 0.7939
Epoch 4/100
935/935 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1004.6022 - mae: 1.1179 - val_loss: 650.7427 - val_mae: 0.9667
Epoch 5/100
935/935 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 705.0059 - mae: 0.9200 - val_loss: 278.5200 - val_mae: 0.9028
Epoch 6/100
935/935 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1700.9478 - mae: 1.1433 - val_loss: 694.9827 - val_mae: 1.1148
Epoch 7/100
935/935 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 790.3846 - mae: 1.0228 - val_loss: 286.0191 - val_mae: 0.9375
Epoch 8/100
935/935 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 877.9879 - mae: 0.9954 - val_loss: 54.2489 - val_mae: 0.7732
Epoch 9/100
9

In [ ]:
RN_Regresion_poly3.evaluate(X_test_poly3,y_test)

292/292 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 14.0884 - mae: 0.9707


[4.8587212562561035, 0.9285630583763123]

#Conclusion

en este Reto se han usado los hiperparametros de learning rate , el numero de neuronas por cada capa el porcentaje de dropout y el numero de parametros que recibe la red neruonal , la funcion build model regresion es la encaegada de recibir estos parametros y construir la red neuronal

1. en el caso de la red neuronal se escogieron las tres variables mas correlacionadas con la variable objetivo
2. se pudo observar que la red neuronal simple con las tres variables sobeajusto soloamente estandarizadas el sobreajuste empezo cuando por encima de las 32 neuronas en cada capa por debajo de 32 neuronas se mantuvo estable el valor del "MAE"
3. se desarrollo un polinomio de grado 2 la red empezo a sobreajustar cuando se ponian mas de 32 neuronas
4. en el polinomio grado 3 sobreajusto desde las 16 neuronas y no es un buen modelo con las varibles que se tiene para la regresion lineal
5. se necesitan mas variables para evitar el sobreajuste y seguir modificando los hiperparametros para ver cual es la mejor opcion para la regresion